In [ ]:
import cv2
import numpy as np
import joblib
from pygame import mixer

#For sound
mixer.init()
sound = mixer.Sound('beep.wav')

# #logistic Regression(99.6)
# classifier = joblib.load('Models/LogReg.gzip')
# #SVM(99.9)
# classifier = joblib.load('Models/svm.gzip')
# #Decesion Tree(97.8)
# classifier = joblib.load('Models/DecesionTree.gzip')
# #Voting Classfier(99.9)
# classifier = joblib.load('Models/VotingClf.gzip')
#Random Forest()
classifier = joblib.load('Models/RandomForest.gzip')


face_cascade = cv2.CascadeClassifier('haar cascade files/haarcascade_frontalface_alt.xml')
leye_cascade = cv2.CascadeClassifier('haar cascade files/haarcascade_lefteye_2splits.xml')
reye_cascade = cv2.CascadeClassifier('haar cascade files/haarcascade_righteye_2splits.xml')
font = cv2.FONT_HERSHEY_SIMPLEX

cap = cv2.VideoCapture(0)

count = 0
a=0
b=0
score=0
while True:
    if count == 0:
        img = cv2.imread("Prashant.jpeg")
    ret, img = cap.read()
    if ret == False:
        break
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray,5,1,1)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        leyes = leye_cascade.detectMultiScale(roi_gray, 1.3, 5)
        for (ex, ey, ew, eh) in leyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
            crop_leye = roi_gray[ey:ey+eh, ex:ex+ew]
            crop_leye = cv2.resize(crop_leye,(80, 80), interpolation = cv2.INTER_CUBIC)
        reyes = reye_cascade.detectMultiScale(roi_gray, 1.3, 5)
        for (ex, ey, ew, eh) in reyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
            crop_reye = roi_gray[ey:ey+eh, ex:ex+ew]
            crop_reye = cv2.resize(crop_reye,(80, 80), interpolation = cv2.INTER_CUBIC)
    

    if(count%7 == 0):
        a = classifier.predict([crop_leye.reshape(80*80)])
        b = classifier.predict([crop_reye.reshape(80*80)])
        
# #         For Random Forest only
#         if a < 0:
#             a = -1
#         else:
#             a = 1
#         if b < 0:
#             b = -1
#         else:
#             b = 1
            
        if ((a==-1 or b==-1) and score<=0):
            score=0
        elif ((a==-1 or b==-1) and score>=3):
            score += (a+b)/4
        elif ((a==-1 or b==-1) and score<3):
            score += (a+b)/3
        else:
            score += (a+b)/2
        if (score>=10):
            score = 10

    tex1 = 'Score: '+ str(int(score))    
    cv2.putText(img,tex1,(10,25), font,1,(255,255,255),1,cv2.LINE_AA)
    if(score>=5):
        cv2.putText(img,"Drowsy Alert!!!",(200,55),font,1,(0,0,255),2,cv2.LINE_AA)
        if(count%5==0):
            if (not((a==-1 or b==-1))):
                try:
                    sound.play()
                except:  # isplaying = False
                    pass
    count += 1
    cv2.imshow("Drowsiness Detection", img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
score

array([6.])